## Setup and Import Libraries

In [2]:
import os
from langchain_neo4j import Neo4jGraph, GraphCypherQAChain
from langchain_groq import ChatGroq
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
graph = Neo4jGraph()

In [6]:
llm = ChatGroq(model_name="Gemma2-9b-It")

## Data Insertion

In [7]:
movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(query=movie_query)

[]

In [8]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {POB: STRING, name: STRING, YOB: INTEGER}
Company {name: STRING}
datascientist {POB: STRING, name: STRING, YOB: INTEGER}
Country {name: STRING}
Person {name: STRING, born: INTEGER}
Movie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Movie)-[:IN_GENRE]->(:Genre)


## Querying The Graph

In [9]:
chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    exclude_types=["Genre"],
    allow_dangerous_requests=True
)

In [10]:
chain.graph_schema

'Node properties:\nCEO {POB: STRING, name: STRING, YOB: INTEGER}\nCompany {name: STRING}\ndatascientist {POB: STRING, name: STRING, YOB: INTEGER}\nCountry {name: STRING}\nPerson {name: STRING, born: INTEGER}\nMovie {title: STRING, released: INTEGER, id: STRING, imdbRating: FLOAT}\nRelationship properties:\n\nThe relationships:\n(:Person)-[:ACTED_IN]->(:Movie)\n(:Person)-[:DIRECTED]->(:Movie)'

In [11]:
query = "How many artists are there?"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(DISTINCT p) AS artistCount
Full Context:
[{'artistCount': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists.  \n'}

In [13]:
query = "How many movies has Tom Hanks acted in?"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[r:ACTED_IN]->(m:Movie) RETURN count(r)
Full Context:
[{'count(r)': 3}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in?',
 'result': "I don't know the answer. \n"}

In [15]:
query = "Which directors have made movies with at least three different actors named 'John'?"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person {name: "John"})
WHERE a.name = "John"
GROUP BY d.name
HAVING COUNT(DISTINCT a) >= 3
RETURN d.name


CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'GROUP': expected an expression, 'FOREACH', 'ORDER BY', 'CALL', 'CREATE', 'LOAD CSV', 'DELETE', 'DETACH', 'FINISH', 'INSERT', 'LIMIT', 'MATCH', 'MERGE', 'NODETACH', 'OFFSET', 'OPTIONAL', 'REMOVE', 'RETURN', 'SET', 'SKIP', 'UNION', 'UNWIND', 'USE', 'WITH' or <EOF> (line 3, column 1 (offset: 101))
"GROUP BY d.name"
 ^}

## Prompt Template

In [36]:
examples = [
    {
        "question": "How many artists are there?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)",
    },
    {
        "question": "Which actors played in the movie Casino?",
        "query": "MATCH (m:Movie {title: 'Casino'})<-[:ACTED_IN]-(a) RETURN a.name",
    },
    {
        "question": "How many movies has Tom Hanks acted in?",
        "query": "MATCH (a:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN count(m)",
    },
    {
        "question": "List all the genres of the movie Toy Story",
        "query": "MATCH (m:Movie {title: 'Toy Story'})-[:IN_GENRE]->(g:Genre) RETURN g.name",
    },
    {
        "question": "Which actors have worked in movies from both the comedy and action genres?",
        "query": "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name",
    },
    {
        "question": "Which directors have made movies with at least three different actors named 'John'?",
        "query": "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name",
    },
    {
        "question": "Identify movies where directors also played a role in the film.",
        "query": "MATCH (p:Person)-[:DIRECTED]->(m:Movie), (p)-[:ACTED_IN]->(m) RETURN m.title, p.name",
    },
    {
        "question": "Find the actor with the highest number of movies in the database.",
        "query": "MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) RETURN a.name, COUNT(m) AS movieCount ORDER BY movieCount DESC LIMIT 1",
    },
]

In [37]:
example_prompt = PromptTemplate.from_template(
    "User input:{question}\n Cypher query:{query}"
)

prompt = FewShotPromptTemplate(
    examples=examples[:3],
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question","schema"]
)

prompt

FewShotPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, examples=[{'question': 'How many artists are there?', 'query': 'MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)'}, {'question': 'Which actors have worked in movies from both the comedy and action genres?', 'query': "MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name"}, {'question': "Which directors have made movies with at least three different actors named 'John'?", 'query': "MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name"}], example_prompt=PromptTemplate(input_variables=['query', 'question'], input_types={}, partial_variables={}, template='User input:{question}\n Cypher query:{query}'), suffix='User input: {question}\nCypher 

In [38]:
print(prompt.format(question="How many artists are there?", schema="foo"))

You are a Neo4j expert. Given an input question,create a syntactically very accurate Cypher query

User input:How many artists are there?
 Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie) RETURN count(DISTINCT a)

User input:Which actors have worked in movies from both the comedy and action genres?
 Cypher query:MATCH (a:Person)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g1:Genre), (a)-[:ACTED_IN]->(:Movie)-[:IN_GENRE]->(g2:Genre) WHERE g1.name = 'Comedy' AND g2.name = 'Action' RETURN DISTINCT a.name

User input:Which directors have made movies with at least three different actors named 'John'?
 Cypher query:MATCH (d:Person)-[:DIRECTED]->(m:Movie)<-[:ACTED_IN]-(a:Person) WHERE a.name STARTS WITH 'John' WITH d, COUNT(DISTINCT a) AS JohnsCount WHERE JohnsCount >= 3 RETURN d.name

User input: How many artists are there?
Cypher query: 


In [39]:
chain = chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    verbose=True,
    cypher_prompt=prompt,
    allow_dangerous_requests=True
)

In [40]:
query = "How many artists are there?"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (a:Person) RETURN count(DISTINCT a)

Full Context:
[{'count(DISTINCT a)': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists.  \n'}

In [41]:
query = "How many movies has Tom Hanks acted in?"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN count(m)

Full Context:
[{'count(m)': 3}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in?',
 'result': "I don't know the answer. \n"}

In [43]:
query = "Which actors acted in the movie Casino?"

response = chain.invoke({"query": query})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title: "Casino"})<-[:ACTED_IN]-(a:Person) RETURN a.name

Full Context:
[{'a.name': 'Robert De Niro'}, {'a.name': 'Joe Pesci'}, {'a.name': 'Sharon Stone'}, {'a.name': 'James Woods'}]

> Finished chain.


{'query': 'Which actors acted in the movie Casino?',
 'result': 'Robert De Niro, Joe Pesci, Sharon Stone, and James Woods acted in the movie Casino.  \n'}